In [9]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
from gensim.models import FastText

In [16]:
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Package omw-1.4 is already up-to-date!
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
replace /kaggle/working/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  /kaggle/working/nltk_data/corpora/omw-1.4.zip
replace /kaggle/working/nltk_data/corpora/omw-1.4/fin/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [17]:
data = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json",lines=True)   

In [11]:
print(data.shape)

(908915, 5)


In [12]:
data

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [18]:
texts=list(data['text'])
sentences=texts[:1500]
print("\nSamples of Sentences\n {}".format(sentences[0:20]))


Samples of Sentences
 ['Avengers time with the ladies.', 'They have lots of good deserts and tasty cuban sandwiches', "It's open even when you think it isn't", 'Very decent fried chicken', 'Appetizers.. platter special for lunch', 'Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.', "Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!", 'This is probably the best place in the cool Springs area to watch a game and eat', 'Tacos', 'Starbucks substitute in boring downtown Tampa. Ugh. Never again!', 'Order the Tortilla Soup', 'Very good will definitely be coming back!!', 'If the Hotlight is on you must stop in.', "Let's go Yankees!", "Basically same food as rally's for $5 more", "Don't go for dinner. They close at 6. Really Yvonne L? Negative stars for a restaurant where you you didn't and never i

In [19]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Tokenization
    words = word_tokenize(text)
    
    # Removing single characters and words with less than three characters
    words = [word for word in words if len(word) > 3]
    
    # Removing punctuation and non-alphanumeric characters
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    
    # Removing stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

In [20]:
processed_sentences = []
for text in sentences:
    processed_sentences.append(preprocess_text(text))

In [21]:
print(processed_sentences[:50])
print(len(processed_sentences))

[['avenger', 'time', 'lady'], ['lot', 'good', 'desert', 'tasty', 'cuban', 'sandwich'], ['open', 'even', 'think'], ['decent', 'fried', 'chicken'], ['appetizer', 'platter', 'special', 'lunch'], ['chili', 'single', 'cheeseburger', 'onion', 'pickle', 'relish', 'vanilla', 'cocacola'], ['saturday', '2013', 'ride', 'patco', 'silver', 'sleigh', 'santa', 'elf', 'decorated', 'train', 'center', 'city', 'train', 'leave', 'lindenwold', '10am', '1115am', '1230pm', 'make', 'stop', 'great', 'kid'], ['probably', 'best', 'place', 'cool', 'spring', 'area', 'watch', 'game'], ['taco'], ['starbucks', 'substitute', 'boring', 'downtown', 'tampa', 'never'], ['order', 'tortilla', 'soup'], ['good', 'definitely', 'coming', 'back'], ['hotlight', 'must', 'stop'], ['yankee'], ['basically', 'food', 'rally'], ['dinner', 'close', 'really', 'yvonne', 'negative', 'star', 'restaurant', 'never', 'intended'], ['min', 'take', 'right', 'right', 'tight'], ['grilled', 'chicken', 'entree', 'delicious', 'small', 'portion', 'like'

In [34]:
# Train FastText model
fast_text_model = FastText(sentences=processed_sentences,vector_size=100, window=5, min_count=1, workers=4, sg=1,epochs=500)


In [35]:
word_embedding = fast_text_model.wv['spring']
print(word_embedding.shape)
print(word_embedding)

(100,)
[ 0.21595459  0.24742496 -0.29892135  0.25301942  0.22249646 -0.65805566
 -0.60807645  0.7266059   0.12013876  0.13535783 -1.171691   -0.50095886
 -1.0381166   0.5152887   0.6228856  -0.43947777 -0.30463466 -1.3550552
  0.73447424 -1.0340949  -1.2012861  -0.37304917 -0.43930304  0.8053189
 -0.8527449  -1.380798    0.89308137  1.0594772   2.3094351  -0.17253609
  0.04719451  0.9529843   0.45335636  0.2522571   0.055135   -0.22101513
  1.0377762   1.0386924  -0.7752276   0.32516143  0.70323133 -0.65874887
  0.7869767  -0.89912677 -0.21539788  1.1565632   0.01148323 -1.2935992
 -0.05244862  0.23918141 -0.03543231  0.51350206  0.27619508  0.00471991
  0.43617636 -0.33642772 -0.28998837 -1.05028     0.49636802  0.17793
  0.6063673  -0.16129711 -0.5730173   2.522606   -0.37227988  0.68740547
  0.30977687  0.03153759  0.09655726  0.9050561   0.10404605  0.83313745
  0.24226971  0.18370269  0.28547987  0.03736012 -0.9036842  -1.251966
 -0.32472396  0.8197575  -0.26808006 -0.07586719 -1.

In [36]:
# Example words
example_words = ['portion', 'spring', 'area', 'watch', 'game', 'eat','allow', 'time', 'wait','chicken','make','like','soup','place', 'great', 'coffee', 'owner','incorrect', 'yelp', 'close']
print(len(example_words))

20


In [37]:
# Print most similar and most dissimilar words for each example word
for word in example_words:
    # Print most similar words to the current example word
    print(f"\nMost similar words to '{word}':")
    similar_words = fast_text_model.wv.most_similar(word, topn=10)
    for sim_word, sim_score in similar_words:
        print(f"{sim_word}: {sim_score:.4f}")  # Print similar word and its similarity score   
    # Print most dissimilar words to the current example word
    print(f"\nMost dissimilar words to '{word}':")
    dissimilar_words = fast_text_model.wv.most_similar(negative=[word], topn=10)
    for dissim_word, dissim_score in dissimilar_words:
        print(f"{dissim_word}: {dissim_score:.4f}")  # Print dissimilar word and its dissimilarity score
    print('------------------------------------------------------')


Most similar words to 'portion':
environment: 0.7765
pimento: 0.7343
rehydration: 0.7306
population: 0.7083
addition: 0.7051
guac: 0.7022
combination: 0.6911
suggestion: 0.6753
figure: 0.6730
altercation: 0.6709

Most dissimilar words to 'portion':
never: -0.1119
deal: -0.1408
empty: -0.1530
come: -0.1578
yelp: -0.1612
philly: -0.1626
downtown: -0.1632
lmfao: -0.1808
amazing: -0.1835
close: -0.1862
------------------------------------------------------

Most similar words to 'spring':
talapia: 0.7324
watercress: 0.6987
roll: 0.6908
watering: 0.6845
panang: 0.6795
goodi: 0.6653
lentil: 0.6643
wedding: 0.6549
boring: 0.6460
appetizersthey: 0.6407

Most dissimilar words to 'spring':
lmfao: -0.0597
dawg: -0.1266
cheese: -0.1634
chocolate: -0.1663
checkout: -0.1726
store: -0.1729
effcient: -0.1738
ever: -0.1758
yelp: -0.1785
even: -0.1793
------------------------------------------------------

Most similar words to 'area':
illusionist: 0.7299
togo: 0.7248
ardmore: 0.7238
artifact: 0.7054
h

Load Pretrained Model

In [26]:
import fasttext
from huggingface_hub import hf_hub_download


In [4]:
model_path = hf_hub_download(repo_id="facebook/fasttext-et-vectors", filename="model.bin")
model = fasttext.load_model(model_path)

In [27]:
model['sport'].shape
model['sport']

array([-0.03365332,  0.05170889,  0.04172005, -0.02595082, -0.07218976,
        0.01548568,  0.0160786 , -0.0667473 ,  0.0964478 , -0.07348094,
       -0.08737053,  0.0213233 , -0.1304598 ,  0.0795631 , -0.08712482,
        0.05686213, -0.05117111, -0.02905971, -0.06115595,  0.03996428,
        0.06802163, -0.01247957, -0.00649385,  0.02979136, -0.06739235,
        0.00070893, -0.02062226, -0.00038709, -0.09722396, -0.04983425,
       -0.01418041, -0.12967938,  0.03733072, -0.02862578,  0.02358723,
       -0.06875183, -0.10802806, -0.02951121,  0.02630502,  0.01353376,
       -0.05284692,  0.00182319, -0.0247485 , -0.03850322, -0.00724931,
       -0.06756096, -0.00891013, -0.09973576, -0.0226414 , -0.02202617,
       -0.06192933, -0.03663646, -0.0367228 , -0.01844403,  0.02946982,
       -0.05029259,  0.02220808, -0.07015549, -0.03860395,  0.03714835,
        0.08684583, -0.00331403,  0.12358376, -0.01328438, -0.00109038,
       -0.06158886,  0.00452175,  0.05010078,  0.03630375, -0.01

In [38]:
# Print most similar and most dissimilar words for each example word
for word in example_words:
    # Print most similar words to the current example word
    print(f"\nMost similar words to '{word}':")
    similar_words = model.get_nearest_neighbors(word, k=10)
    for sim_word, sim_score in similar_words:
        print(f"{sim_word}: {sim_score}")  # Print similar word and its similarity score

    # Print most dissimilar words to the current example word
    print(f"\nMost dissimilar words to '{word}':")
    similar_words = model.get_nearest_neighbors(word, k=len(model.words))
    # Sort the similar_words list by similarity score
    similar_words_sorted = sorted(similar_words, key=lambda x: x[1], reverse=True)
    for sim_word, sim_score in similar_words[-10:]:
        print(f"{sim_word}: {sim_score}")  # Print similar word and its similarity score


Most similar words to 'portion':
0.8117364645004272: proportion
0.7681371569633484: Funtion
0.7617917060852051: portions
0.7575868964195251: 5tion
0.7555277943611145: Proportion
0.7459480166435242: assertion
0.739467978477478: decryption
0.7388937473297119: encryption
0.7385836243629456: Extortion
0.7375932931900024: portio

Most dissimilar words to 'portion':
-0.4602525234222412: b7
-0.46264931559562683: Beb
-0.46287137269973755: 丹陽郡
-0.4653022289276123: žw
-0.4660171866416931: 富平區
-0.4668874740600586: Öp
-0.4704243838787079: 珍島郡
-0.47148939967155457: Goé
-0.4762401282787323: Hõ
-0.4848965108394623: TERG

Most similar words to 'spring':
0.8124701380729675: IRPaleFormspring
0.8115647435188293: offspring
0.7699934244155884: Handspring
0.7564191222190857: springform
0.7540384531021118: springs
0.7389662265777588: Offspring
0.7131842970848083: springar
0.6853605508804321: springi
0.6769934892654419: springer
0.6751344799995422: sprint201004

Most dissimilar words to 'spring':
-0.38157072